In [69]:
from pyoshub import OSH_API
from dotenv import load_dotenv
import os
import pandas as pd
import json
import csv

In [5]:
load_dotenv()  # take environment variables from .env.
OSH_URL = os.environ.get("OSH_URL")
OSH_TOKEN = os.environ.get("OSH_TOKEN")

In [23]:
osh_api = OSH_API(url=OSH_URL, token=OSH_TOKEN)

In [29]:
facilities_by_page = osh_api.get_facilities(page=1, pageSize=1)

In [35]:
facilities_by_page

NameError: name 'facilities_by_page' is not defined

In [31]:
len(facilities_by_page)

100

In [32]:
type(facilities_by_page[0])

dict

In [34]:
facilities_df = pd.DataFrame(facilities_by_page)

In [35]:
facilities_df.head()

,os_id,lon,lat,name,address,country_code,country_name,has_approved_claim,is_closed
0,CN2021250D1DTN7,119.278740,33.786210,"Huai An Yuan Tong Headwear Mfg. Co., Ltd.","No.30 & 32 & 99 Yan Huang Avenue, Lian Shui Ec...",CN,China,False,None
1,BD201915877ME2J,90.383062,24.087520,HABITUS FASHION LIMITED,Gajaria Para Bahwal Mirzapur Gazipur Sadar Dha...,BD,Bangladesh,False,None
2,CN2019083DCWXWP,121.673980,29.857010,KASHION INDUSTRY CO. LTD,"555 MEIDISI ROAD, WUXIANG INDUSTRY PARK, YINZH...",CN,China,True,None
3,TR2019091T7CABY,28.000589,41.282927,Bony Tekstil Isletmeleri A.S.,Cerkezköy Organize Sanayi Bölgesi (OSB) Ismet ...,TR,Türkiye,False,None
4,BD2019248W1R8FC,90.394233,24.226893,Crown Exclusive Wears Ltd.,"Crown Exclusive Wears Ltd. Building, Soling Mo...",BD,Bangladesh,False,None


In [37]:
facilities_df.columns

Index(['os_id', 'lon', 'lat', 'name', 'address', 'country_code',
       'country_name', 'has_approved_claim', 'is_closed'],
      dtype='object')

In [39]:
facilities_df["lat"]

0     33.786210
1     24.087520
2     29.857010
3     41.282927
4     24.226893
        ...    
95    22.997358
96    23.707795
97    11.497719
98    29.977913
99    30.230027
Name: lat, Length: 100, dtype: float64

In [40]:
facilities_df.dtypes

os_id                  object
lon                   float64
lat                   float64
name                   object
address                object
country_code           object
country_name           object
has_approved_claim       bool
is_closed              object
dtype: object

In [43]:
type(facilities_df['os_id'].iloc[0])

str

In [44]:
facilities_df.to_csv("output-data/facilities.csv")

In [142]:
input_data_path = 'input-data'
file_list = os.listdir(input_data_path)

In [223]:
all_json_data = []

for filename in file_list:
    if filename.endswith('.json'):
        input_file_path = os.path.join(input_data_path, filename)
        with open(input_file_path, 'r') as file:
            input_data = json.load(file)
            all_json_data.extend(input_data)

In [224]:
# not necessary but stores json as backup 
with open("input-data/all_json_data.json", "w") as file:
    json.dump(all_json_data, file)

In [225]:
osh_df = pd.json_normalize(all_json_data, max_level=1)

In [226]:
osh_df.columns

Index(['id', 'type', 'geometry.type', 'geometry.coordinates',
       'properties.name', 'properties.address', 'properties.country_code',
       'properties.os_id', 'properties.country_name',
       'properties.has_approved_claim', 'properties.ppe_product_types',
       'properties.ppe_contact_phone', 'properties.ppe_contact_email',
       'properties.ppe_website', 'properties.is_closed'],
      dtype='object')

In [227]:
osh_df.to_csv('output-data/osh-data.csv')

In [228]:
osh_df.head(2)


,id,type,geometry.type,geometry.coordinates,properties.name,properties.address,properties.country_code,properties.os_id,properties.country_name,properties.has_approved_claim,properties.ppe_product_types,properties.ppe_contact_phone,properties.ppe_contact_email,properties.ppe_website,properties.is_closed
0,PK20211592H3207,Feature,Point,"[73.0317104, 31.4003816]",Umer Faizan Enterprises,"Jhang Road, Saifabad Moar, Naimatabad Street N...",PK,PK20211592H3207,Pakistan,False,None,None,None,None,None
1,LK201908506Y46Y,Feature,Point,"[80.0372819, 7.4006552]",Unichela (Pvt) Ltd.- Slimline Division,"Kuliyapitiya Road, Pannala, Kurunegala Pannala",LK,LK201908506Y46Y,Sri Lanka,False,None,None,None,None,None


In [229]:
# split geometry coordinates to lat and long like facilities.csv
osh_df['lon'] = osh_df['geometry.coordinates'].apply(lambda x: x[0])
osh_df['lat'] = osh_df['geometry.coordinates'].apply(lambda x: x[1])


In [230]:
osh_df

,id,type,geometry.type,geometry.coordinates,properties.name,properties.address,properties.country_code,properties.os_id,properties.country_name,properties.has_approved_claim,properties.ppe_product_types,properties.ppe_contact_phone,properties.ppe_contact_email,properties.ppe_website,properties.is_closed,lon,lat
0,PK20211592H3207,Feature,Point,"[73.0317104, 31.4003816]",Umer Faizan Enterprises,"Jhang Road, Saifabad Moar, Naimatabad Street N...",PK,PK20211592H3207,Pakistan,False,None,None,None,None,None,73.031710,31.400382
1,LK201908506Y46Y,Feature,Point,"[80.0372819, 7.4006552]",Unichela (Pvt) Ltd.- Slimline Division,"Kuliyapitiya Road, Pannala, Kurunegala Pannala",LK,LK201908506Y46Y,Sri Lanka,False,None,None,None,None,None,80.037282,7.400655
2,LK2019108BDMFGQ,Feature,Point,"[80.220935, 7.274062]",Unichela (Pvt) Ltd - Thurulie Division,"Mas Fabric Park, Kurunegala Road, Thulhiriya, ...",LK,LK2019108BDMFGQ,Sri Lanka,False,None,None,None,None,None,80.220935,7.274062
3,VN2019083DDV9F8,Feature,Point,"[104.8423858, 21.701618]",Unico Global YB,"KCN Au Lau, Xa Au Lau, Thanh pho Yen Bai, Tinh...",VN,VN2019083DDV9F8,Vietnam,False,"[Scrubs, coveralls]",,park.ys@unicoglobal.com,,None,104.842386,21.701618
4,BD2021090RDB5X6,Feature,Point,"[90.1612029, 24.099259]",Uniglory Packaging Industries,"Gorai, Mirzapur, Tangail, Dhaka, Bangladesh",BD,BD2021090RDB5X6,Bangladesh,False,None,None,None,None,None,90.161203,24.099259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,CN2019172QEMX30,Feature,Point,"[119.619985, 29.845502]","Hangzhou Tonglu Tiancheng Knitting Co., Ltd","No. 158, Xianghe Road Hangzhou 311512 Zhejiang",CN,CN2019172QEMX30,China,False,None,None,None,None,None,119.619985,29.845502
4996,CN2019083PD3XHW,Feature,Point,"[119.691434, 29.793535]",Hangzhou Yutu Spinning&Knitting Co Ltd,Tonglu County Hangzhou Zhejiang Province,CN,CN2019083PD3XHW,China,False,None,None,None,None,None,119.691434,29.793535
4997,CN2020053NVNG2M,Feature,Point,"[120.168123, 30.4499147]","Hangzhou Zhongtian Dyeing & Weaving Co., Ltd","5 SHUNQI RD., TANGQI TOWN, YUHANG Hangzhou Zhe...",CN,CN2020053NVNG2M,China,False,None,None,None,None,None,120.168123,30.449915
4998,CN2019176GCEVDX,Feature,Point,"[119.606017, 29.828431]",Hangzhou Zhongyi Textile Fashion,"No. 458, Hengfu Road, Hengcun Town Tonglu Coun...",CN,CN2019176GCEVDX,China,False,None,None,None,None,None,119.606017,29.828431


In [231]:
# rename columns to match csv
rename_columns = {'id': 'os_id', 'properties.name': 'name', 'properties.address': 'address', 'properties.country_name': 'country_name', 'properties.country_code': 'country_code', 'properties.country_name': 'country_name', 'properties.is_closed' : 'is_closed', 'properties.has_approved_claim': 'has_approved_claim'}
osh_df.rename(columns=rename_columns, inplace=True)

In [232]:
#drop osh ones we dont care about
drop_columns = ['type', 'geometry.type', 'geometry.coordinates', 'properties.os_id', 'properties.ppe_product_types', 'properties.ppe_contact_phone', 'properties.ppe_contact_email', 'properties.ppe_website']
osh_to_merge = osh_df.drop(drop_columns, axis=1)


In [233]:
osh_to_merge.head(2)
osh_to_merge.to_csv('output-data/osh-data-sanitized.csv')

In [234]:
# import both files
df1 = pd.read_csv('./output-data/facilities.csv')
df2 = pd.read_csv('./output-data/osh-data-sanitized.csv')

In [235]:
# match columns to df1
df2 = df2[df1.columns]

In [241]:
combined_df = pd.concat([df1, df2], ignore_index=False)
combine_df_sanitized = combined_df.drop('Unnamed: 0', axis=1)

In [243]:
combine_df_sanitized.drop_duplicates(subset=['os_id'], keep='first')

,os_id,lon,lat,name,address,country_code,country_name,has_approved_claim,is_closed
0,CN2021250D1DTN7,119.278740,33.786210,"Huai An Yuan Tong Headwear Mfg. Co., Ltd.","No.30 & 32 & 99 Yan Huang Avenue, Lian Shui Ec...",CN,China,False,NaN
1,BD201915877ME2J,90.383062,24.087520,HABITUS FASHION LIMITED,Gajaria Para Bahwal Mirzapur Gazipur Sadar Dha...,BD,Bangladesh,False,NaN
2,CN2019083DCWXWP,121.673980,29.857010,KASHION INDUSTRY CO. LTD,"555 MEIDISI ROAD, WUXIANG INDUSTRY PARK, YINZH...",CN,China,True,NaN
3,TR2019091T7CABY,28.000589,41.282927,Bony Tekstil Isletmeleri A.S.,Cerkezköy Organize Sanayi Bölgesi (OSB) Ismet ...,TR,Türkiye,False,NaN
4,BD2019248W1R8FC,90.394233,24.226893,Crown Exclusive Wears Ltd.,"Crown Exclusive Wears Ltd. Building, Soling Mo...",BD,Bangladesh,False,NaN
...,...,...,...,...,...,...,...,...,...
4995,CN2019172QEMX30,119.619985,29.845502,"Hangzhou Tonglu Tiancheng Knitting Co., Ltd","No. 158, Xianghe Road Hangzhou 311512 Zhejiang",CN,China,False,NaN
4996,CN2019083PD3XHW,119.691434,29.793535,Hangzhou Yutu Spinning&Knitting Co Ltd,Tonglu County Hangzhou Zhejiang Province,CN,China,False,NaN
4997,CN2020053NVNG2M,120.168123,30.449915,"Hangzhou Zhongtian Dyeing & Weaving Co., Ltd","5 SHUNQI RD., TANGQI TOWN, YUHANG Hangzhou Zhe...",CN,China,False,NaN
4998,CN2019176GCEVDX,119.606017,29.828431,Hangzhou Zhongyi Textile Fashion,"No. 458, Hengfu Road, Hengcun Town Tonglu Coun...",CN,China,False,NaN


In [244]:
combine_df_sanitized.head(2)

combine_df_sanitized.to_csv('./output-data/merged-data.csv')